<a href="https://colab.research.google.com/github/PRASAD212019/GenerativeAI/blob/main/RAG_Advanced_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1 - Overview of embeddings-based retrieval

Welcome! Here's a few notes about the Chroma course notebooks.
 - A number of warnings pop up when running the notebooks. These are normal and can be ignored.
 - Some operations such as calling an LLM or an opeation using generated data return unpredictable results and so your notebook outputs may differ from the video.
  
Enjoy the course!

In [ ]:
!pip install pypdf langchain openai transformers chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.5 MB/s

# Basic RAG

In [ ]:
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return (string[:n_chars].rsplit(' ', 1)[0] + '\n' +
                word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars))

In [ ]:
from pypdf import PdfReader

reader = PdfReader("/content/microsoft_annual_report_2022.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

In [ ]:
print(word_wrap(pdf_texts[0][0:1000]))

1 Dear shareholders, colleagues, customers, and partners:  
We are
living through a period of historic economic, societal, and
geopolitical change. The world in 2022 looks nothing like 
the world in
2019. As I write this, inflation is at a 40 -year high, supply chains
are stretched, and the war in Ukraine is 
ongoing. At the same time, we
are entering a technological era with the potential to power awesome
advancements 
across every sector of our economy and society. As the
world’s largest software company, this places us at a historic

intersection of opportunity and responsibility to the world around us.
 
Our mission to empower every person and every organization on the
planet to achieve more has never been more 
urgent or more necessary.
For all the uncertainty in the world, one thing is clear: People and
organizations in every 
industry are increasingly looking to digital
technology to overcome today’s challenges and emerge stronger. And no

company is better positioned to help th

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [ ]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=100
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

In [ ]:
#print(word_wrap(character_split_texts[10]))
#print(f"\nTotal chunks: {len(character_split_texts)}")

In [ ]:
#token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

#token_split_texts = []
#for text in character_split_texts:
#    token_split_texts += token_splitter.split_text(text)

#print(word_wrap(token_split_texts[10]))
#print(f"\nTotal chunks: {len(token_split_texts)}")

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
print(embedding_function([character_split_texts[10]]))

[[0.04256267845630646, 0.033211849629879, 0.03034009225666523, -0.034866586327552795, 0.06841650605201721, -0.08090908825397491, -0.015474384650588036, -0.001450937008485198, -0.016744472086429596, 0.06770772486925125, -0.05054135248064995, -0.04919537156820297, 0.05139991268515587, 0.09192730486392975, -0.07177839428186417, 0.039519693702459335, -0.012833530083298683, -0.02494746446609497, -0.04622865840792656, -0.02435753121972084, 0.03394966572523117, 0.02550245262682438, 0.027317119762301445, -0.0041262200102210045, -0.03633834794163704, 0.0036908735055476427, -0.027430471032857895, 0.004796730354428291, -0.028896236792206764, -0.018870729953050613, 0.03666628524661064, 0.02569585107266903, 0.03131284937262535, -0.0639343410730362, 0.05394405871629715, 0.08225349336862564, -0.04175683856010437, -0.00699579669162631, -0.02348603680729866, -0.030747927725315094, -0.0029792017303407192, -0.07790938764810562, 0.009353099390864372, 0.003162842011079192, -0.022257033735513687, -0.0182946

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022",
                                                    embedding_function=embedding_function)

ids = [str(i) for i in range(len(character_split_texts))]

chroma_collection.add(ids=ids, documents=character_split_texts)
chroma_collection.count()

354

In [ ]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=[query], n_results=3)
retrieved_documents = results['documents'][0]

In [ ]:
for document in retrieved_documents:
    print('===========================================================')
    print(word_wrap(document))
    print('\n')

impracticability of determining the geographic source of the revenue. 

Revenue, classified by significant product and service offerings, was
as follows:  
  
(In millions)          
        
Year Ended June  30,
2022  2021  2020  
        
Server products and cloud services  $
67,321   $ 52,589   $ 41,379   
Office products and cloud services  
44,862    39,872    35,316   
Windows   24,761    22,488    21,510  

Gaming   16,230    15,370    11,575   
LinkedIn   13,816    10,289   
8,077   
Search and news advertising   11,591    9,267    8,524  

Enterprise Services   7,407    6,943    6,409   
Devices   6,991   
6,791    6,457   
Other   5,291    4,479    3,768         
Total  $  
198,270   $  168,088   $   143,015           
We have recast certain
previously reported amounts in the table above to conform to the way we
internally manage and 
monitor our business.


74   
NOTE 13 — UNEARNED REVENUE  
Unearned revenue by segment was as
follows:  
  
(In millions)        
      
June  

In [ ]:
import os
import openai
from openai import OpenAI
from getpass import getpass

In [ ]:
openai.api_key = getpass("OpenAI Key: ")

OpenAI Key: ··········


In [ ]:
openai_client = OpenAI(api_key = openai.api_key)

In [ ]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print(word_wrap(output))

The total revenue for the year ended June 30, 2022, was $198,270
million.


# Query 2

**What were the most important factors that contributed to increases in revenue?**


In [ ]:
original_query = "What were the most important factors that contributed to increases in revenue?"

results_q2 = chroma_collection.query(query_texts=[original_query], n_results=3)
retrieved_documents_q2 = results_q2['documents'][0]

In [ ]:
output_q2 = rag(query=original_query, retrieved_documents=retrieved_documents_q2)

print(word_wrap(output_q2))

The most important factors that contributed to increases in revenue
were commercial sales. Additionally, investments in cloud engineering,
Gaming, LinkedIn, and commercial sales drove the increase in research
and development expenses and sales and marketing expenses. The growth
in Windows OEM and Windows Commercial also contributed to the increase
in revenue.


## Query Expansion

In [ ]:
def augment_multiple_query(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about an annual report. "
            "Suggest two additional related questions to help them find the information they need, for the provided question. "
            "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
            "Make sure they are complete questions, and that they are related to the original question."
            "Output one question per line. Do not number the questions."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content

In [ ]:
augmented_queries = augment_multiple_query(original_query)

In [ ]:
for query in augmented_queries:
    print(query)

What were the major sources of revenue growth?
What strategies were implemented to drive revenue growth?


In [ ]:
queries = [original_query] + augmented_queries
results = chroma_collection.query(query_texts=queries, n_results=2, include=['documents', 'embeddings'])

retrieved_documents = results['documents']

# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

In [ ]:
for i, documents in enumerate(retrieved_documents):
    print(f"Query: {queries[i]}")
    print('')
    print("Results:")
    for doc in documents:
        print(word_wrap(doc))
        print('')
    print('-'*100)

Query: What were the most important factors that contributed to increases in revenue?

Results:
commercial sales.  
Key changes in operating expenses were:  
•
Research and development expenses increased $3.8  billion or 18% driven
by investments in cloud 
engineering, Gaming, and LinkedIn.  
• Sales
and marketing expenses increased $1.7  billion or 8% driven by
investments in commercial sales and 
LinkedIn. Sales and marketing
included a favorable foreign currency impact of 2%.  
• General and
administrative expenses increased $793  million or 16% driven by
investments in corporate 
functions.  
Operating income increased $13.5
 billion or 19% driven by growth across each of our segments.  
Current
year net income and diluted EPS were positively impacted by the net tax
benefit related to the transfer of 
intangible properties, which
resulted in an increase to net income and diluted EPS of $3.3  billion
and $0.44, respectively. 
Prior year net income and diluted EPS were
positively imp

In [ ]:
output = rag(query=query, retrieved_documents=unique_documents)

print(word_wrap(output))

The strategies implemented to drive revenue growth included investments
in cloud engineering, Gaming, LinkedIn, and commercial sales. These
investments contributed to an increase in sales and marketing expenses
by $1.7 billion or 8%. Additionally, the revenue growth was driven by
growth across each of the segments, including Intelligent Cloud revenue
increased by Azure and other cloud services, Productivity and Business
Processes revenue increased by Office 365 Commercial and LinkedIn, and
More Personal Computing revenue increased by Search and news
advertising and Windows.


## Document Retriever with Cross Encoder Re-ranking

In [ ]:
query_3 = "What has been the investment in research and development?"
results_3 = chroma_collection.query(query_texts=query_3, n_results=10, include=['documents', 'embeddings'])

In [ ]:
retrieved_documents_3 = results_3['documents'][0]

In [ ]:
for document in results_3['documents'][0]:
    print('===========================================================')
    print(word_wrap(document))
    print('')

Gross margin percentage was relatively unchanged.  
• Operating
expenses increased $1.5  billion or 14% driven by investments in
Gaming, Search and news 
advertising, and Windows marketing. 

OPERATING EXPENSES  
Research and Development  
  
(In millions,
except percentages)  2022  2021  Percentage  
Change    
       

Research and development  $   24,512    $  20,716     18%  
As a
percent of revenue   12%   12%   0ppt    
Research and development
expenses include payroll, employee benefits, stock -based compensation
expense, and other 
headcount -related expenses associated with product
development. Research and development expenses also include

third-party development and programming costs, localization costs
incurred to translate software for international markets, 
and the
amortization of purchased software code and services content. 

Research and development expenses increased $3.8  billion or 18%
driven by investments in cloud engineering, Gaming, 
and LinkedIn.

research an

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
pairs = [[query_3, doc] for doc in retrieved_documents_3]
scores = cross_encoder.predict(pairs)

In [ ]:
scores

array([  0.67515856,   3.64366   , -10.731592  ,   1.6427071 ,
        -7.7066054 ,  -5.646997  ,  -4.297037  , -11.017633  ,
        -5.8040953 ,  -7.003495  ], dtype=float32)

In [ ]:
import pandas as pd

In [ ]:
doc_with_scores = pd.DataFrame( { "doc" : retrieved_documents_3,
                                  "score": scores })

In [ ]:
doc_with_scores = doc_with_scores.sort_values("score", ascending = False).reset_index()
doc_with_scores

,index,doc,score
0,1,research and development facilities in other p...,3.643660
1,3,products. \nInvesting in the Future \nOur su...,1.642707
2,0,Gross margin percentage was relatively unchang...,0.675159
3,6,but generally include parts and labor over a p...,-4.297037
4,5,"At times, we make select intellectual property...",-5.646997
5,8,"cash flows will occur, and determination of ou...",-5.804095
6,9,documentation internally. \nWe protect our in...,-7.003495
7,4,also increased the number of identified partne...,-7.706605
8,2,when the world around us does well. \nThat’s ...,-10.731592
9,7,and customers around the world use Microsoft t...,-11.017633


In [ ]:
doc_with_scores['doc'][0:3]

0    research and development facilities in other p...
1    products.  \nInvesting in the Future  \nOur su...
2    Gross margin percentage was relatively unchang...
Name: doc, dtype: object

In [ ]:
output = rag(query=query_3, retrieved_documents=doc_with_scores['doc'][0:3])

In [ ]:
print(word_wrap(output))

The investment in research and development for the year is $24,512
million, which represents an 18% increase compared to the previous
year.
